In [266]:
from glob import glob
import json
import pathlib
from datetime import datetime, timezone, timedelta
from operator import itemgetter, attrgetter
import tempfile
import os


In [7]:
SORA_BASE_DIR = "/mnt/data/sora_video/"

このコードの目的
- SORAで記録された動画を、ロボットが動いていた時間だけにカットする
- ロボット稼働時間用のカットツールは https://github.com/nkawa/Metawork_log_analysis

In [25]:
import pandas as pd

In [431]:
allwork = pd.read_csv("2025-08-25_09-07.allwork.csv",index_col=0)

In [433]:
len(allwork)

475

In [434]:
# アプリ名と省略名、channel-id の対応
APP_TABLE = {
    '/withDualCam/puzzle': ("puzzle",["uclab-vr180","uclab-hand"]),
    '/withDualCam/candy':  ("candy",["uclab-vr180","uclab-hand"]),
    '/jaka/practice/'   : ("jakaPractice",[]),
    '/jaka/withDualCam/': ("jakaPuzzle",["paichi-vr180","paichi-hand"])
}

In [435]:
assets = glob(SORA_BASE_DIR+"*")

In [436]:
# 各フォルダのビデオを調査
def check_files(assets= assets):
    finfo = []
    JST = timezone(timedelta(hours=9))

    for d in assets:
        p = pathlib.Path(d)
        # 各フォルダの情報ファイルを確認
        subinfo = []
        for x in glob(d+"/split-archive-*.json"):
                if "-end-" in x:
                    continue
                with open(x,"r",encoding="utf-8") as f:
                    js2 = json.load(f)
                    startdt = datetime.strptime(js2["start_timestamp"][:-1]+"+00:00","%Y-%m-%dT%H:%M:%S.%f%z").astimezone(JST)
                    enddt = datetime.strptime(js2["stop_timestamp"][:-1]+"+00:00","%Y-%m-%dT%H:%M:%S.%f%z").astimezone(JST)
                    fobj = {
                        "fname": x,
                        "vfile": x[:-4]+"mp4",
                        "channel_id": js2["channel_id"],
                        "startTime": startdt.timestamp(),
                        "startStr":  str(startdt),
                        "endTime":enddt.timestamp(),
                        "duration": enddt.timestamp()-startdt.timestamp()
                    }
                    subinfo.append(fobj)       
        subinfo = sorted(subinfo, key=itemgetter("startTime"))
        for x in glob(d+"/report-*.json"): # 基本は１つだけのはず
            with open(x,"r",encoding="utf-8") as f:
                js = json.load(f)
                startdt = datetime.strptime(js["start_timestamp"][:-1]+"+00:00","%Y-%m-%dT%H:%M:%S.%f%z").astimezone(JST)
                enddt = datetime.strptime(js["stop_timestamp"][:-1]+"+00:00","%Y-%m-%dT%H:%M:%S.%f%z").astimezone(JST)
                fobj = {
                    "fname": x,
                    "channel_id": js["channel_id"],
                    "startTime": startdt.timestamp(),
                    "startStr":  str(startdt),
                    "endTime":enddt.timestamp(),
                    "duration": enddt.timestamp()-startdt.timestamp(),
                    "subs": subinfo
                }
                finfo.append(fobj)
#                print(js)
    finfo = sorted(finfo, key=itemgetter("startTime"))
    return finfo

In [437]:
# 特定のmp4 ファイルを、必要なだけ切り取るコマンド
#
def do_ffmpeg_command_start_duration_out(infile, start, duration, outfile):
    if duration == 0:
        sout = "ffmpeg -ss {0} -i {1} -c:v copy {2}".format(start, infile,outfile)
    else:
        sout = "ffmpeg -ss {0} -t {1} -i {2} -c:v copy {3}".format(start, duration,infile,outfile)       
    rt = os.system(sout)
    print(rt,sout)
    return sout

# concat demuxer を使うため、テンポラルファイルを作成
def do_ffmpeg_concat(infile_1, infile_2, outfile):
    with tempfile.NamedTemporaryFile(delete=True) as t:
        with open(t.name, "w+") as f:
            f.write("file '"+infile_1+"'\n")
            f.write("file '"+infile_2+"'\n")
        cmd = "ffmpeg -f concat -safe 0 -i {0} -c copy {1}".format(t.name,outfile )
        rt = os.system(cmd)
        print(rt,cmd)


In [438]:
finfo = check_files()

In [439]:
len(finfo)

73

In [186]:
finfo[0]

{'fname': '/mnt/data/sora_video/JGJKQKG4RX0EX6V43JT3FJWTBC/report-JGJKQKG4RX0EX6V43JT3FJWTBC.json',
 'channel_id': 'paichi-hand',
 'startTime': 1756204455.673,
 'startStr': '2025-08-26 19:34:15.673000+09:00',
 'endTime': 1756204460.424,
 'duration': 4.750999927520752,
 'subs': [{'fname': '/mnt/data/sora_video/JGJKQKG4RX0EX6V43JT3FJWTBC/split-archive-7A1RTKW6ED72X3KA0YGHMWRCA8_0001.json',
   'vfile': '/mnt/data/sora_video/JGJKQKG4RX0EX6V43JT3FJWTBC/split-archive-7A1RTKW6ED72X3KA0YGHMWRCA8_0001.mp4',
   'channel_id': 'paichi-hand',
   'startTime': 1756204455.713632,
   'startStr': '2025-08-26 19:34:15.713632+09:00',
   'endTime': 1756204458.413632,
   'duration': 2.6999998092651367}]}

In [321]:
# まずは 2025-09-01 のpuzzle だけやろう
alltasks = allwork[(allwork["app"] == "/withDualCam/puzzle")&(allwork["date"] == "2025-09-01")&(allwork["count"]>=1000)].to_dict(orient="records")

In [338]:
alltasks = allwork[(allwork["count"]>=1000)].to_dict(orient="records")

In [461]:
alltasks = allwork[(allwork["date"] == "2025-09-05")&(allwork["count"]>=1000)].to_dict(orient="records")

In [462]:
len(alltasks)

42

In [243]:
alltasks = allwork[(allwork["wid"] == "4744aad8-a031-7068-058a-1083e231ccaf")&(allwork["app"] == "/withDualCam/puzzle")&(allwork["count"]>=1000)].to_dict(orient="records")

In [179]:
alltasks[0]

{'id': '新城VR3-w3ort6r',
 'date': '2025-08-27',
 'fname': 'mqtt-recorder-control-新城VR3-w3ort6r-20250827-183132-0.json',
 'startTime': 1756287092.2713046,
 'startStr': '2025-08-27 18:31:32',
 'endStr': '18:41:34',
 'endTime': 1756287694.4225671,
 'count': 19655,
 'device': "{'agent': 'Mozilla/5.0 (X11; Linux x86_64; Quest 3) AppleWebKit/537.36 (KHTML, like Gecko) OculusBrowser/40.1.0.6.64.776937802 Chrome/138.0.7204.179 VR Safari/537.36', 'cookie': True}",
 'wid': '4744aad8-a031-7068-058a-1083e231ccaf',
 'app': '/withDualCam/puzzle',
 'regTime': '2025-08-27 18:31:27.259043'}

In [161]:
alltasks = allwork.to_dict(orient="records")

In [442]:
# task に対応するビデオを探して、分割してフォルダにおく
# ファイル命名規則  タスク種別/channel_id/日付//wid_task_日付.mp4
DEST_FOLDER = "/mnt/data/task_clip"
def crop_video(task, finfo=finfo):
    use_files= []
    wid = task["wid"]
    (app,chan) = APP_TABLE.get(task["app"],("none",[]))
    date = task["date"]
    st = task["startTime"]
    ed = task["endTime"]
    sub = None
    if app == "jakaPractice": # そもそも practice は対象としない
        return use_files
    for x in finfo:
        if x["channel_id"] in chan:
            if x["startTime"]<= st and x["endTime"] >= ed:
                outfile = DEST_FOLDER+"/{0}/{1}/{2}/{3}_{0}_{1}_{2}.mp4".format(app,x["channel_id"],date,wid)
                pt = pathlib.Path(outfile)
                os.makedirs(str(pt.parent), exist_ok=True)
#                print("Find finfo", task["startStr"], task["endStr"], x["startStr"],x["duration"],x["channel_id"],app)
                subs = x["subs"]
                for i, s in enumerate(subs):
                    if s["startTime"]<= st and st <=s["endTime"]: #スタートが間にある
                        if s["endTime"] >= ed:  #1つの動画でカバーできた
#                        print("Find subinfo",s["vfile"])
#                       print(s["startStr"],s["duration"])
                            #単独の場合は、単純に ffmpeg 呼び出しでOK
                            sub = s["vfile"]
                            use_files.append(sub)
                            do_ffmpeg_command_start_duration_out(sub, st-s["startTime"], ed-st, outfile)
                            break
                        elif subs[i+1]["endTime"] >= ed: # ２つのファイルに動画が分割されている場合
                            print("Find Connected subinfo",s["vfile"], subs[i+1]["vfile"])
#                        print(s["startStr"],s["duration"],subs[i+1]["startStr"],subs[i+1]["duration"])
                            out_sub1 =  outfile[:-4]+"_1.mp4"
                            out_sub2 =  outfile[:-4]+"_2.mp4"
                            sub = s["vfile"]
                            do_ffmpeg_command_start_duration_out(sub, st-s["startTime"],0, out_sub1)
                            do_ffmpeg_command_start_duration_out(subs[i+1]["vfile"], 0, ed-subs[i+1]["startTime"], out_sub2)
                            use_files.append(sub)
                            use_files.append(sub)
                            do_ffmpeg_concat(out_sub1,out_sub2,outfile)
                            # remove
                            os.remove(out_sub1)
                            os.remove(out_sub2)                            
                            break 
    if sub == None:
        print("Can't find proper subinfo",date, app, task["startStr"],task["endStr"])
    return use_files
                
        
    

In [330]:
# タスクに関わったファイルだけ集めたい

In [443]:
# alltask から使ったファイルを全部取得
def check_all_used():
    used = []
    allnum = len(alltasks)
    for i,t in enumerate(alltasks):
        print(i,"/",allnum,datetime.now())
        used.extend(crop_video(t))
    return list(dict.fromkeys(used))


In [444]:
# 全部のsubfile を作る
def all_subfile():
    subs = []
    for x in finfo:
        sb = x["subs"]
        for s in sb:
            subs.append(s["vfile"])
    return subs

In [363]:
all_subs = all_subfile()

In [406]:
len(alltasks)

46

In [ ]:
used = check_all_used()

0 / 30 2025-09-08 19:15:28.671839


ffmpeg version 4.4.2-0ubuntu0.22.04.1 Copyright (c) 2000-2021 the FFmpeg developers
  built with gcc 11 (Ubuntu 11.2.0-19ubuntu1)
  configuration: --prefix=/usr --extra-version=0ubuntu0.22.04.1 --toolchain=hardened --libdir=/usr/lib/x86_64-linux-gnu --incdir=/usr/include/x86_64-linux-gnu --arch=amd64 --enable-gpl --disable-stripping --enable-gnutls --enable-ladspa --enable-libaom --enable-libass --enable-libbluray --enable-libbs2b --enable-libcaca --enable-libcdio --enable-libcodec2 --enable-libdav1d --enable-libflite --enable-libfontconfig --enable-libfreetype --enable-libfribidi --enable-libgme --enable-libgsm --enable-libjack --enable-libmp3lame --enable-libmysofa --enable-libopenjpeg --enable-libopenmpt --enable-libopus --enable-libpulse --enable-librabbitmq --enable-librubberband --enable-libshine --enable-libsnappy --enable-libsoxr --enable-libspeex --enable-libsrt --enable-libssh --enable-libtheora --enable-libtwolame --enable-libvidstab --enable-libvorbis --enable-libvpx --enab

0 ffmpeg -ss 1232.6444447040558 -t 253.93652391433716 -i /mnt/data/sora_video/DYK1WSK8RX55N3PE2B8EDQC184/split-archive-YHF9FQ2MWX2KDB67YKQP60P0K4_0001.mp4 -c:v copy /mnt/data/task_clip/puzzle/uclab-vr180/2025-09-07/4744aad8-a031-7068-058a-1083e231ccaf_puzzle_uclab-vr180_2025-09-07.mp4


Input #0, mov,mp4,m4a,3gp,3g2,mj2, from '/mnt/data/sora_video/0R9VWND1ZH1KVE5ZVVAFHH1XWM/split-archive-AH8FVZAQ813995NGRGJ8J0YK7G_0001.mp4':
  Metadata:
    major_brand     : isom
    minor_version   : 0
    compatible_brands: isomiso2mp41iso6
    creation_time   : 2025-09-07T00:55:26.000000Z
  Duration: 00:59:40.13, start: 0.000000, bitrate: 842 kb/s
  Stream #0:0(und): Video: hevc (Main) (hev1 / 0x31766568), yuv420p(tv), 640x480 [SAR 1:1 DAR 4:3], 838 kb/s, 29.90 fps, 29.97 tbr, 1000k tbn, 30 tbc (default)
    Metadata:
      rotate          : 0
      creation_time   : 2025-09-07T00:55:26.000000Z
      handler_name    : VideoHandler
      vendor_id       : [0][0][0][0]
    Side data:
      displaymatrix: rotation of -0.00 degrees
Output #0, mp4, to '/mnt/data/task_clip/puzzle/uclab-hand/2025-09-07/4744aad8-a031-7068-058a-1083e231ccaf_puzzle_uclab-hand_2025-09-07.mp4':
  Metadata:
    major_brand     : isom
    minor_version   : 0
    compatible_brands: isomiso2mp41iso6
    encoder   

0 ffmpeg -ss 1232.73392868042 -t 253.93652391433716 -i /mnt/data/sora_video/0R9VWND1ZH1KVE5ZVVAFHH1XWM/split-archive-AH8FVZAQ813995NGRGJ8J0YK7G_0001.mp4 -c:v copy /mnt/data/task_clip/puzzle/uclab-hand/2025-09-07/4744aad8-a031-7068-058a-1083e231ccaf_puzzle_uclab-hand_2025-09-07.mp4
1 / 30 2025-09-08 19:15:37.633153
256 ffmpeg -ss 648.1645178794861 -t 473.2867066860199 -i /mnt/data/sora_video/DYK1WSK8RX55N3PE2B8EDQC184/split-archive-YHF9FQ2MWX2KDB67YKQP60P0K4_0001.mp4 -c:v copy /mnt/data/task_clip/puzzle/uclab-vr180/2025-09-07/4744aad8-a031-7068-058a-1083e231ccaf_puzzle_uclab-vr180_2025-09-07.mp4
256 ffmpeg -ss 648.2540018558502 -t 473.2867066860199 -i /mnt/data/sora_video/0R9VWND1ZH1KVE5ZVVAFHH1XWM/split-archive-AH8FVZAQ813995NGRGJ8J0YK7G_0001.mp4 -c:v copy /mnt/data/task_clip/puzzle/uclab-hand/2025-09-07/4744aad8-a031-7068-058a-1083e231ccaf_puzzle_uclab-hand_2025-09-07.mp4
2 / 30 2025-09-08 19:15:37.714691


...

ffmpeg version 4.4.2-0ubuntu0.22.04.1 Copyright (c) 2000-2021 the FFmpeg developers
  built with gcc 11 (Ubuntu 11.2.0-19ubuntu1)
  configuration: --prefix=/usr --extra-version=0ubuntu0.22.04.1 --toolchain=hardened --libdir=/usr/lib/x86_64-linux-gnu --incdir=/usr/include/x86_64-linux-gnu --arch=amd64 --enable-gpl --disable-stripping --enable-gnutls --enable-ladspa --enable-libaom --enable-libass --enable-libbluray --enable-libbs2b --enable-libcaca --enable-libcdio --enable-libcodec2 --enable-libdav1d --enable-libflite --enable-libfontconfig --enable-libfreetype --enable-libfribidi --enable-libgme --enable-libgsm --enable-libjack --enable-libmp3lame --enable-libmysofa --enable-libopenjpeg --enable-libopenmpt --enable-libopus --enable-libpulse --enable-librabbitmq --enable-librubberband --enable-libshine --enable-libsnappy --enable-libsoxr --enable-libspeex --enable-libsrt --enable-libssh --enable-libtheora --enable-libtwolame --enable-libvidstab --enable-libvorbis --enable-libvpx --enab

0 ffmpeg -ss 3525.262093067169 -i /mnt/data/sora_video/0R9VWND1ZH1KVE5ZVVAFHH1XWM/split-archive-AH8FVZAQ813995NGRGJ8J0YK7G_0007.mp4 -c:v copy /mnt/data/task_clip/puzzle/uclab-hand/2025-09-07/3744aad8-d041-703d-9c1b-4e96edb0070e_puzzle_uclab-hand_2025-09-07_1.mp4


frame=17888 fps=7887 q=-1.0 Lsize=   62916kB time=00:09:57.08 bitrate= 863.2kbits/s speed= 263x    
video:62745kB audio:0kB subtitle:0kB other streams:0kB global headers:0kB muxing overhead: 0.272890%
ffmpeg version 4.4.2-0ubuntu0.22.04.1 Copyright (c) 2000-2021 the FFmpeg developers
  built with gcc 11 (Ubuntu 11.2.0-19ubuntu1)
  configuration: --prefix=/usr --extra-version=0ubuntu0.22.04.1 --toolchain=hardened --libdir=/usr/lib/x86_64-linux-gnu --incdir=/usr/include/x86_64-linux-gnu --arch=amd64 --enable-gpl --disable-stripping --enable-gnutls --enable-ladspa --enable-libaom --enable-libass --enable-libbluray --enable-libbs2b --enable-libcaca --enable-libcdio --enable-libcodec2 --enable-libdav1d --enable-libflite --enable-libfontconfig --enable-libfreetype --enable-libfribidi --enable-libgme --enable-libgsm --enable-libjack --enable-libmp3lame --enable-libmysofa --enable-libopenjpeg --enable-libopenmpt --enable-libopus --enable-libpulse --enable-librabbitmq --enable-librubberband --e

0 ffmpeg -ss 0 -t 597.0964548587799 -i /mnt/data/sora_video/0R9VWND1ZH1KVE5ZVVAFHH1XWM/split-archive-AH8FVZAQ813995NGRGJ8J0YK7G_0008.mp4 -c:v copy /mnt/data/task_clip/puzzle/uclab-hand/2025-09-07/3744aad8-d041-703d-9c1b-4e96edb0070e_puzzle_uclab-hand_2025-09-07_2.mp4
0 ffmpeg -f concat -safe 0 -i /tmp/tmpmu3s44mf -c copy /mnt/data/task_clip/puzzle/uclab-hand/2025-09-07/3744aad8-d041-703d-9c1b-4e96edb0070e_puzzle_uclab-hand_2025-09-07.mp4
21 / 30 2025-09-08 19:21:14.866637


ffmpeg version 4.4.2-0ubuntu0.22.04.1 Copyright (c) 2000-2021 the FFmpeg developers
  built with gcc 11 (Ubuntu 11.2.0-19ubuntu1)
  configuration: --prefix=/usr --extra-version=0ubuntu0.22.04.1 --toolchain=hardened --libdir=/usr/lib/x86_64-linux-gnu --incdir=/usr/include/x86_64-linux-gnu --arch=amd64 --enable-gpl --disable-stripping --enable-gnutls --enable-ladspa --enable-libaom --enable-libass --enable-libbluray --enable-libbs2b --enable-libcaca --enable-libcdio --enable-libcodec2 --enable-libdav1d --enable-libflite --enable-libfontconfig --enable-libfreetype --enable-libfribidi --enable-libgme --enable-libgsm --enable-libjack --enable-libmp3lame --enable-libmysofa --enable-libopenjpeg --enable-libopenmpt --enable-libopus --enable-libpulse --enable-librabbitmq --enable-librubberband --enable-libshine --enable-libsnappy --enable-libsoxr --enable-libspeex --enable-libsrt --enable-libssh --enable-libtheora --enable-libtwolame --enable-libvidstab --enable-libvorbis --enable-libvpx --enab

In [376]:
not_used = list(set(all_subs)-set(used))
not_used.sort()

In [381]:
not_used[0]

'/mnt/data/sora_video/05NXYY6NFH3CV8ZN3DQ9M8MD40/split-archive-CYE14QKRAS2FV8TR28R46RY9AW_0001.mp4'

In [379]:
len(used)

204

In [380]:
# not_used の時刻を調べよう

In [394]:
def check_not_used(not_used = not_used):
    for f in not_used:
        jsf = f[:-4]+".json"
        with open(jsf,"r",encoding="utf-8") as f:
            js2 = json.load(f)
            startdt = datetime.strptime(js2["start_timestamp"][:-1]+"+00:00","%Y-%m-%dT%H:%M:%S.%f%z").astimezone(JST)
            enddt = datetime.strptime(js2["stop_timestamp"][:-1]+"+00:00","%Y-%m-%dT%H:%M:%S.%f%z").astimezone(JST)
            print(startdt, js2["channel_id"])

In [392]:
not_used.sort()

In [395]:
check_not_used()

2025-08-26 19:34:28.439353+09:00 uclab-vr180
2025-08-26 20:34:08.578665+09:00 uclab-vr180
2025-09-03 18:49:22.601823+09:00 paichi-hand
2025-08-26 19:34:28.421028+09:00 paichi-vr180
2025-08-26 20:34:08.614561+09:00 paichi-vr180
2025-09-05 09:11:59.092752+09:00 uclab-vr180
2025-09-05 10:11:39.311229+09:00 uclab-vr180
2025-09-05 11:11:39.448990+09:00 uclab-vr180
2025-08-29 11:52:46.058436+09:00 uclab-hand
2025-08-29 14:52:46.614566+09:00 uclab-hand
2025-08-29 18:52:47.441948+09:00 uclab-hand
2025-08-29 19:52:47.638215+09:00 uclab-hand
2025-09-04 13:00:15.428485+09:00 paichi-hand
2025-09-04 13:59:55.577463+09:00 paichi-hand
2025-09-04 14:59:55.781536+09:00 paichi-hand
2025-09-04 15:59:55.952642+09:00 paichi-hand
2025-09-04 16:59:56.126434+09:00 paichi-hand
2025-09-04 17:59:56.345313+09:00 paichi-hand
2025-09-02 19:01:25.314818+09:00 uclab-hand
2025-09-02 20:01:25.516964+09:00 uclab-hand
2025-09-02 21:01:25.710784+09:00 uclab-hand
2025-09-02 22:01:25.833801+09:00 uclab-hand
2025-09-02 23:01

In [418]:
1

1

In [424]:
for x in alltasks:
    print(x["date"],x["startStr"])

2025-08-30 2025-08-30 09:07:36
2025-08-30 2025-08-30 09:15:04
2025-08-30 2025-08-30 09:33:36
2025-08-30 2025-08-30 10:14:11
2025-08-30 2025-08-30 10:30:06
2025-08-30 2025-08-30 10:48:39
2025-08-30 2025-08-30 11:11:36
2025-08-30 2025-08-30 11:40:55
2025-08-30 2025-08-30 11:22:21
2025-08-30 2025-08-30 13:53:02
2025-08-30 2025-08-30 13:59:05
2025-08-30 2025-08-30 15:18:02
2025-08-30 2025-08-30 15:43:57
2025-08-30 2025-08-30 15:25:24
2025-08-30 2025-08-30 16:17:23
2025-08-30 2025-08-30 16:27:14
2025-08-30 2025-08-30 17:14:58
2025-08-30 2025-08-30 17:30:12
2025-08-30 2025-08-30 18:12:48
2025-08-30 2025-08-30 18:20:48
